## using backbone from pre-trained keras resnet50/vgg19 apps

In [1]:
%matplotlib inline
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.applications.resnet50 import *
from tensorflow.keras.datasets import cifar10
# from keras.datasets import cifar10
# from keras.applications.resnet50 import *
# from keras.layers import *
# from keras.models import *
# import keras

import numpy as np
import pandas as pd
keras.__version__

'2.2.4-tf'

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 5903s 35us/step


### let's train a bin classifier, find what happens when pos/neg samples are extremely unbalanced

In [3]:
x_train_bin = x_train.copy()
y_train_bin = y_train==0
x_train_bin.shape, y_train_bin.shape

((50000, 32, 32, 3), (50000, 1))

In [4]:
x = preprocess_input(x_train_bin[0])

In [5]:
x = np.expand_dims(x, axis=0)
x.shape

(1, 32, 32, 3)

In [8]:
m = ResNet50(weights='imagenet', include_top=False)


In [9]:
for layer in m.layers:
    layer.trainable = False

In [10]:
x_features = m.predict(x=x)
x_features.shape

(1, 1, 1, 2048)

In [11]:
x = m.output
x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

In [12]:
baseline_model = Model(inputs=m.input, outputs=prediction)
baseline_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
______________________________________________________________________________________________

In [13]:
baseline_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

In [14]:
x_valid_bin = x_test.copy()
y_valid_bin = y_test==1

In [15]:
import pandas as pd
hist = baseline_model.fit(x=x_train_bin, y=y_train_bin, validation_data=(x_valid_bin, y_valid_bin), epochs=10, 
                          verbose=1)

W0717 09:47:54.533415 140223054829376 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 46s 912us/sample - loss: 0.2610 - acc: 0.9028 - val_loss: 0.5286 - val_acc: 0.8878
Epoch 2/10
50000/50000 [==============================] - 41s 813us/sample - loss: 0.2298 - acc: 0.9123 - val_loss: 0.6157 - val_acc: 0.8915
Epoch 3/10
50000/50000 [==============================] - 41s 821us/sample - loss: 0.2207 - acc: 0.9143 - val_loss: 0.5154 - val_acc: 0.8819  - ETA: 12 - ETA - ETA: 10s - loss: 0.224 - ETA: 9s - loss: 0. - ETA: 6s - loss: 0.2225 -  -  - ETA: - ETA: 0s - loss: 0.2211 - acc: 0. - ETA: 0s - loss: 0.2209 - acc:  - ETA: 0s - loss: 0.2208 - acc: 0.91 - ETA: 0s - loss: 0.2207 - acc: 0.914
Epoch 4/10
50000/50000 [==============================] - 41s 826us/sample - loss: 0.2156 - acc: 0.9163 - val_loss: 0.4949 - val_acc: 0.8697- ETA: 6s - loss: 0.2157 - acc: 0.916  - ETA: 1s - loss: 0.2156 - acc: 0 - ETA: 1s - loss: 0
Epoch 5/10
50000/50000 [==========================

In [16]:
baseline_model.evaluate(x=x_valid_bin, y=y_valid_bin)

10000/10000 [==============================] - 7s 654us/sample - loss: 0.4759 - acc: 0.8614


[0.4758749816417694, 0.8614]

### train the bin classifier with skewed data

In [17]:
feats = m.output

In [18]:
x = GlobalAveragePooling2D()(feats)
x = Dense(10, activation='relu')(x)
y = Dense(1, activation='sigmoid')(x)
test_model = Model(inputs=m.inputs, outputs=y)

In [19]:
test_model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])

In [20]:
mask = (y_train==1)
mask = np.squeeze(mask)
mask.shape

(50000,)

In [21]:
x_train_bin_sess1 = x_train_bin[mask]
x_train_bin_sess1.shape

(5000, 32, 32, 3)

In [22]:
y_train_bin_sess1 = y_train_bin[mask]
y_train_bin_sess1.shape

(5000, 1)

In [23]:
test_model.fit(x=x_train_bin_sess1, y=y_train_bin_sess1, epochs=10, verbose=1)

Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 6s 1ms/sample - loss: 0.0441 - acc: 0.9942
Epoch 2/10
5000/5000 [==============================] - 3s 692us/sample - loss: 0.0073 - acc: 1.0000s - loss: 0.0087 - acc: 1.000 - ETA: 1s -
Epoch 3/10
5000/5000 [==============================] - 3s 690us/sample - loss: 0.0040 - acc: 1.0000
Epoch 4/10
5000/5000 [==============================] - 3s 690us/sample - loss: 0.0029 - acc: 1.0000s - loss: 
Epoch 5/10
5000/5000 [==============================] - 3s 690us/sample - loss: 0.0022 - acc: 1.0000
Epoch 6/10
5000/5000 [==============================] - 3s 690us/sample - loss: 0.0020 - acc: 1.0000
Epoch 7/10
5000/5000 [==============================] - 3s 690us/sample - loss: 0.0015 - acc: 1.0000
Epoch 8/10
5000/5000 [==============================] - 3s 689us/sample - loss: 0.0014 - acc: 1.0000
Epoch 9/10
5000/5000 [==============================] - 3s 689us/sample - loss: 0.0012 - acc: 1.0000
Epoch 10/10
5000/500

In [24]:
test_model.evaluate(x=x_valid_bin, y=y_valid_bin)

10000/10000 [==============================] - 8s 835us/sample - loss: 1.5291 - acc: 0.9000


[1.5290701082196272, 0.9]

Appearantly, that wont work since this model is only taught by possitive examples. An auto negative example gens seems to be necessary. Just keep stream in negative samples and see what happens

In [25]:
# this time stream in data with 2nd class
mask = y_train==2
mask = np.squeeze(mask)
x_train_bin_sess2 = x_train_bin[mask]
y_train_bin_sess2 = y_train_bin[mask]
mask.shape, x_train_bin_sess2.shape, y_train_bin_sess2.shape

((50000,), (5000, 32, 32, 3), (5000, 1))

In [26]:
test_model.fit(x=x_train_bin_sess2, y=y_train_bin_sess2, epochs=10, verbose=1)

Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 3s 691us/sample - loss: 0.0012 - acc: 1.0000s - loss: 0.0012 - acc: 1.000 - ETA: 0s - loss: 0.0012 - a
Epoch 2/10
5000/5000 [==============================] - 3s 688us/sample - loss: 0.0010 - acc: 1.0000
Epoch 3/10
5000/5000 [==============================] - 3s 688us/sample - loss: 0.0011 - acc: 1.0000
Epoch 4/10
5000/5000 [==============================] - 3s 688us/sample - loss: 9.4801e-04 - acc: 1.0000
Epoch 5/10
5000/5000 [==============================] - 3s 688us/sample - loss: 8.6727e-04 - acc: 1.0000
Epoch 6/10
5000/5000 [==============================] - 3s 689us/sample - loss: 7.8012e-04 - acc: 1.0000
Epoch 7/10
5000/5000 [==============================] - 3s 689us/sample - loss: 7.0309e-04 - acc: 1.0000
Epoch 8/10
5000/5000 [==============================] - 3s 690us/sample - loss: 6.8939e-04 - acc: 1.0000
Epoch 9/10
5000/5000 [==============================] - 3s 687us/sample - loss: 6.1559e-04 -

In [27]:
testout = test_model.predict(x_valid_bin)

The model simply goes to the other extreme end

In [28]:
np.sum(testout>0.1)

0

Even in bin classifier, skewed data hurt model badly

## Train the bin classifier incrementally

Next, we create 'perfect' learning sessions manually trying to defeat the baseline model (89% valid acc). The purpose of this experiment is to figure out whether extra negtive samples help with the model

In [29]:
def get_model_bin_classify(feature_extractor):
    x = feature_extractor.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=feature_extractor.input, outputs=x)
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])
    return model

In [30]:
def get_train_samples(X_train, y_train, cls_li):
    mask = np.zeros(y_train.shape, dtype=np.bool)
    for cls in cls_li:
        mask += (y_train==cls)
    mask = np.squeeze(mask)
    return X_train[mask], y_train[mask]
    
def eva_bin_on_cls(model, X_valid, y_valid, cls_li, pos_samples):
    for cls in cls_li:
        mask = (y_valid==cls)
        mask = np.squeeze(mask)
        X_valid_cur_sess = X_valid[mask]
        y_valid_cur_sess = y_valid[mask]
        y_valid_cur_sess = convert_to_bin_samples(y_valid_cur_sess, pos_samples)
        loss, metric = model.evaluate(x=X_valid_cur_sess, y=y_valid_cur_sess)
        print('Evaluation loss: %f. Acc: %f' % (loss, metric))

def convert_to_bin_samples(labels, pos_samples):
    # arg1 = original labels, pos_samples = samples mark possitive
    re_label = np.zeros(labels.shape, dtype=np.bool)
    for it in pos_samples:
        re_label += (labels == it)
    return re_label

In [31]:
feat_ext = ResNet50(weights='imagenet', include_top=False)

In [32]:
test_model = get_model_bin_classify(feat_ext)

In [33]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 1])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 3s 3ms/sample - loss: 0.1582 - acc: 0.9440
Evaluation loss: 0.158220. Acc: 0.944000
1000/1000 [==============================] - 1s 671us/sample - loss: 0.0690 - acc: 0.9760
Evaluation loss: 0.068989. Acc: 0.976000


In [34]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 2])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 661us/sample - loss: 0.3202 - acc: 0.8870
Evaluation loss: 0.320182. Acc: 0.887000
1000/1000 [==============================] - 1s 662us/sample - loss: 1.0665 - acc: 0.7170
Evaluation loss: 1.066504. Acc: 0.717000
1000/1000 [==============================] - 1s 667us/sample - loss: 0.1755 - acc: 0.9520
Evaluation loss: 0.175457. Acc: 0.952000


In [35]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 3])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 662us/sample - loss: 0.4150 - acc: 0.8980
Evaluation loss: 0.415038. Acc: 0.898000
1000/1000 [==============================] - 1s 664us/sample - loss: 0.8853 - acc: 0.7880
Evaluation loss: 0.885310. Acc: 0.788000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.3662 - acc: 0.9150
Evaluation loss: 0.366227. Acc: 0.915000
1000/1000 [==============================] - 1s 662us/sample - loss: 0.0668 - acc: 0.9720
Evaluation loss: 0.066786. Acc: 0.972000


In [36]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 4])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 663us/sample - loss: 0.1913 - acc: 0.9570
Evaluation loss: 0.191287. Acc: 0.957000
1000/1000 [==============================] - 1s 667us/sample - loss: 2.0515 - acc: 0.5550
Evaluation loss: 2.051499. Acc: 0.555000
1000/1000 [==============================] - 1s 664us/sample - loss: 0.9107 - acc: 0.8280
Evaluation loss: 0.910684. Acc: 0.828000
1000/1000 [==============================] - 1s 665us/sample - loss: 0.3285 - acc: 0.9060
Evaluation loss: 0.328513. Acc: 0.906000
1000/1000 [==============================] - 1s 662us/sample - loss: 0.0933 - acc: 0.9690
Evaluation loss: 0.093305. Acc: 0.969000


In [37]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 5])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 663us/sample - loss: 0.1989 - acc: 0.9540
Evaluation loss: 0.198906. Acc: 0.954000
1000/1000 [==============================] - 1s 666us/sample - loss: 2.2351 - acc: 0.5770
Evaluation loss: 2.235072. Acc: 0.577000
1000/1000 [==============================] - 1s 665us/sample - loss: 0.9296 - acc: 0.8310
Evaluation loss: 0.929616. Acc: 0.831000
1000/1000 [==============================] - 1s 664us/sample - loss: 0.1465 - acc: 0.9610
Evaluation loss: 0.146471. Acc: 0.961000
1000/1000 [==============================] - 1s 665us/sample - loss: 0.2407 - acc: 0.9310
Evaluation loss: 0.240733. Acc: 0.931000
1000/1000 [==============================] - 1s 665us/sample - loss: 0.0843 - acc: 0.9780
Evaluation loss: 0.084341. Acc: 0.978000


In [38]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 6])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5, 6], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 661us/sample - loss: 0.2586 - acc: 0.9440
Evaluation loss: 0.258557. Acc: 0.944000
1000/1000 [==============================] - 1s 662us/sample - loss: 1.7192 - acc: 0.6660
Evaluation loss: 1.719180. Acc: 0.666000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.8885 - acc: 0.8520
Evaluation loss: 0.888452. Acc: 0.852000
1000/1000 [==============================] - 1s 664us/sample - loss: 0.1725 - acc: 0.9540
Evaluation loss: 0.172473. Acc: 0.954000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.2651 - acc: 0.9330
Evaluation loss: 0.265078. Acc: 0.933000
1000/1000 [==============================] - 1s 667us/sample - loss: 0.1174 - acc: 0.9740
Evaluation loss: 0.117410. Acc: 0.974000
1000/1000 [==============================] - 1s 664us/sample - loss: 0.0337 - acc: 0.9930
Evaluation loss: 0.033669. Acc: 0.993000


In [39]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 7])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5, 6, 7], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 661us/sample - loss: 0.3985 - acc: 0.9190
Evaluation loss: 0.398534. Acc: 0.919000
1000/1000 [==============================] - 1s 666us/sample - loss: 1.2722 - acc: 0.7330
Evaluation loss: 1.272159. Acc: 0.733000
1000/1000 [==============================] - 1s 664us/sample - loss: 0.7624 - acc: 0.8610
Evaluation loss: 0.762422. Acc: 0.861000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.0963 - acc: 0.9760
Evaluation loss: 0.096325. Acc: 0.976000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.1363 - acc: 0.9680
Evaluation loss: 0.136346. Acc: 0.968000
1000/1000 [==============================] - 1s 664us/sample - loss: 0.0438 - acc: 0.9880
Evaluation loss: 0.043817. Acc: 0.988000
1000/1000 [==============================] - 1s 662us/sample - loss: 0.0514 - acc: 0.9800
Evaluation loss: 0.051430. Acc: 0.980000
1000/1000 

In [40]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 8])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5, 6, 7, 8], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 664us/sample - loss: 0.4520 - acc: 0.8950
Evaluation loss: 0.451995. Acc: 0.895000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.3210 - acc: 0.8990
Evaluation loss: 0.321037. Acc: 0.899000
1000/1000 [==============================] - 1s 666us/sample - loss: 1.4928 - acc: 0.6810
Evaluation loss: 1.492800. Acc: 0.681000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.5116 - acc: 0.8530
Evaluation loss: 0.511559. Acc: 0.853000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.6470 - acc: 0.8250
Evaluation loss: 0.646994. Acc: 0.825000
1000/1000 [==============================] - 1s 663us/sample - loss: 0.4135 - acc: 0.8760
Evaluation loss: 0.413486. Acc: 0.876000
1000/1000 [==============================] - 1s 662us/sample - loss: 0.3950 - acc: 0.8750
Evaluation loss: 0.394960. Acc: 0.875000
1000/1000 

In [41]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 9])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=3)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0])

Train on 10000 samples
Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5
1000/1000 [==============================] - 1s 671us/sample - loss: 0.3907 - acc: 0.9160
Evaluation loss: 0.390695. Acc: 0.916000
1000/1000 [==============================] - 1s 673us/sample - loss: 0.1490 - acc: 0.9600
Evaluation loss: 0.149020. Acc: 0.960000
1000/1000 [==============================] - 1s 675us/sample - loss: 1.9310 - acc: 0.6210
Evaluation loss: 1.930993. Acc: 0.621000
1000/1000 [==============================] - 1s 677us/sample - loss: 0.4611 - acc: 0.8810
Evaluation loss: 0.461053. Acc: 0.881000
1000/1000 [==============================] - 1s 676us/sample - loss: 0.6122 - acc: 0.8390
Evaluation loss: 0.612185. Acc: 0.839000
1000/1000 [==============================] - 1s 677us/sample - loss: 0.2509 - acc: 0.9300
Evaluation loss: 0.250886. Acc: 0.930000
1000/1000 [==============================] - 1s 678us/sample - loss: 0.2287 - acc: 0.9350
Evaluation loss: 0.228665. Acc: 0.935000
1000/1000 

The result is rather satisfied yet confusing. Why the perf on 2nd class differs by turns? Let's see how a partial trained bin classifier perform on unseened data

## Experiments on unseen negative samples

In [42]:
X_train_sess, y_train_sess = get_train_samples(x_train, y_train, [0, 1])
y_train_sess = convert_to_bin_samples(y_train_sess, [0])
test_model.fit(x=X_train_sess, y=y_train_sess, epochs=5, verbose=1)
eva_bin_on_cls(test_model, x_test, y_test, [0, 1, 2, 3, 4, 5], [0])

Train on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 27s 3ms/sample - loss: 0.0260 - acc: 0.9908
Epoch 2/5
10000/10000 [==============================] - 27s 3ms/sample - loss: 0.0072 - acc: 0.9978
Epoch 3/5
10000/10000 [==============================] - 27s 3ms/sample - loss: 0.0045 - acc: 0.9986
Epoch 4/5
10000/10000 [==============================] - 27s 3ms/sample - loss: 0.0023 - acc: 0.9996
Epoch 5/5
1000/1000 [==============================] - 1s 674us/sample - loss: 0.3652 - acc: 0.9310
Evaluation loss: 0.365189. Acc: 0.931000
1000/1000 [==============================] - 1s 676us/sample - loss: 0.0794 - acc: 0.9830
Evaluation loss: 0.079423. Acc: 0.983000
1000/1000 [==============================] - 1s 678us/sample - loss: 2.8667 - acc: 0.5330
Evaluation loss: 2.866716. Acc: 0.533000
1000/1000 [==============================] - 1s 676us/sample - loss: 0.7207 - acc: 0.8320
Evaluation loss: 0.720703. Acc: 0.832000
1000/1000 [============================

In [43]:
X_test_subset, Y_test_set = get_train_samples(x_test, y_test, [2])
prediction = test_model.predict(X_test_subset)

In [44]:
np.sum(prediction < 0.9) / np.squeeze(prediction).shape[0]

0.621

The model **does not generate itself well enough on unseen negative samples** even by increasing the threshold up to 0.9, yet it will **degrade its performance on positive samples sufficiently**. By which, every/relavent classifiers need to be updated by turns.

## conclusion
- Performance on seen negative samples will be intefered when newly arrived negative samples observed by model. Sometimes increasing sometimes decreasing. 
- It seems like that the model's generation capability is performing better and better when enough negative classes have been trained before.
- The model performs poorly on unseen negative samples which emphasizes the necessity of upgrade when new negative class came